In [3]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
import pandas as pd

https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb

upload data

In [4]:
norm_data = pd.read_csv('../data/normalized_expr_mat.csv', index_col=0)

In [8]:
labels = pd.read_csv('../data/cell_type.csv', index_col=0)
labels.type.unique().tolist()

['5-6',
 '7-1',
 '7-4',
 '1-1',
 '1-2',
 '5-2',
 'MG',
 '5-3',
 '3-5',
 '3-2',
 '2-5',
 '4-2',
 '5-4',
 '6-2',
 '2-4']

In [9]:
norm_data = norm_data.join(labels)

In [10]:
X = norm_data.iloc[:,:-1] # data
Y = norm_data.iloc[:,-1:] # labels

Train test split

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [29]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [30]:
est = model.fit(
    x_train, y_train,
    eval_set=(x_test, y_test),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

TO DO: cross validation

In [32]:
predictions = model.predict(x_test)
predictions_probs = model.predict_proba(x_test)
print(predictions[:10])
print(predictions_probs[:10])

[['7-4']
 ['1-2']
 ['MG']
 ['5-3']
 ['1-1']
 ['2-4']
 ['7-4']
 ['2-4']
 ['7-4']
 ['MG']]
[[6.36151980e-04 1.08529746e-03 4.30815943e-04 2.17810136e-04
  7.13601776e-03 1.46134262e-04 1.00230259e-04 6.24992181e-04
  2.12221895e-03 1.04945669e-04 3.49332581e-03 7.17213153e-04
  2.95271759e-03 9.79731369e-01 5.00759767e-04]
 [9.70296460e-03 9.82921196e-01 1.77844156e-03 2.37894275e-04
  1.90594992e-03 2.30300606e-04 9.26298178e-05 5.58628598e-04
  1.45503573e-04 1.95534974e-04 1.71777049e-04 3.15404405e-04
  3.05155625e-04 1.07375124e-03 3.64867299e-04]
 [1.00463427e-02 4.56769799e-03 1.60433032e-03 5.18986903e-02
  1.77098702e-03 1.16765702e-02 2.77011929e-03 5.75088566e-04
  1.24043424e-02 1.65747360e-03 6.99676658e-04 5.81338144e-03
  7.57051093e-03 5.40881912e-02 8.32856597e-01]
 [1.45109048e-03 9.33512435e-04 5.39471385e-04 9.15792721e-04
  3.99236636e-03 1.28811866e-03 3.38969900e-03 2.33331705e-03
  9.30717996e-01 5.52361084e-03 8.72484006e-03 2.22801577e-02
  7.05501407e-03 7.0577